In [ ]:
from datascience import *
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Lab 10: Shocks
In this lab, we want to look at how different events influence outcomes.

### 0. Reading in Data
We'll start by reading in our data set. This data has information on a series of events that occured within the child's life between the ages of 8 and 12.

In [ ]:
data = Table.read_table('...Country....csv')
data

### 1. Exploring the Data / Summary Statistics

<font color="blue"> Item 1a: Find what portion of children experienced `flooding`.

<font color="blue">Item 1b: What portion of children experienced `mother_ill`?

### 2. Shocks

Which shock do you think has the greatest effect on time for homework? 

<font color="blue"> Item 2: Check if time spent on study at age 12 is lower for children who have experienced this shock. By how much? </font> Bonus: Do the confidence intervals overlap? *Remember to remove the `nan` values from the columns you'll be using.*

### 3. A Longitudinal Perspective

It may be however, that kids that study less are more likely to experience a disaster. That would imply that they also studied less when they were 8. To attempt to control for that, let's look at the change in the hours studied, and if there is a relationship there.  Start by making a new column for the change in hours studied from 8 to 12.  (Remember to remove nans.)

In [ ]:
change_in_hours = data.column('...12...') - data.column('...8...')
data = data.with_column('change from 8 to 12', change_in_hours)
data

<font color="blue"> Item 3:  By how much does the change in hours studied differ for those who have experienced your shock of interest.

Now let's control for wealth as well, using the wealth index at age 8. To do this, we need to change the shocks into numeric form (use `float`).

*Hint: If you create an array of booleans, you can convert it to be an array of floats by using `.astype(float)`.*

In [ ]:
event_numeric = (change_cleaned.column('...shock...') == 'yes').astype(float)
change_cleaned = change_cleaned.with_column('...shock...', event_numeric)
change_cleaned

<font color="blue">Item 4: Do a regression with change in hours studied as the output, and your shock and wealth at age 8 as inputs. Does this change your conclusion in Item 3?</font>

First, make sure that there are non `nans` in your age 8 wealth index column.

In [ ]:
cleaned_inputs = ....where(np.logical_and(....column('wi_8') == ....column('wi_8'),
                            ....column('...shock...') == ....column('...shock...')))

We are going to be using `statsmodels`, a package that includes a lot of regression functionality and displays the results with a nice interface. This will allow us to take advantage of the work that has been done by the larger Python community (and save us from some extra work). The cell below will import the package.

In [ ]:
import statsmodels.api as sm

`statsmodels` regresssion models take arrays as inputs. So assign `outcome` to an array containing our outcome variable. Assign predictors to be an array of the predictors (after you select the columns you want, you can convert a table to a multi-dimensional array by using `.to_array()`.

In [ ]:
outcome = cleaned_inputs.column('...change in study from 8 to 12...')
predictors = cleaned_inputs.select('...shock...', 'wi_8').to_array()

We'll then pass `outcome` and `predictors` into our regression model. No need to change anything in the following cell, just run it!

In [ ]:
results = sm.OLS(outcome, predictors).fit()
results.summary()

<font color="blue"> Item 5: repeat the regression, but include all the shocks.  What other shocks seem important for schooling?  Does your original shock (still) matter? </font> 

We first need to change all of the other shocks into floats and filter out the `nan`s. Use a for loop to do this!

In [ ]:
other_shocks = make_array("birth_hh_member", "cropfailure", ...., "theft")
multiple_shocks_table = cleaned_inputs

for column_name in other_shocks:
    multiple_shocks_table = multiple_shocks_table.where(multiple_shocks_table.column(column_name) == multiple_shocks_table.column(column_name))
    converted_values = (multiple_shocks_table.column(column_name) == 'yes').astype(float)
    multiple_shocks_table = multiple_shocks_table.with_column(column_name, converted_values)
multiple_shocks_table

Use the cell below to get your outcome and predictors into the correct format for our regression model.

In [ ]:
outcome = multiple_shocks_table.column('...change in study from 8 to 12...')
predictors = multiple_shocks_table.select(np.append(other_shocks, make_array('drought', 'wi_8'))).to_array()

Run the cell below to compute the regression parameters.

In [ ]:
results = sm.OLS(outcome, predictors).fit()
results.summary()

<font color="blue">Item 6: Does this change in hours study also translate into a worse vocabulary score? What do you conclude?</font>

Make a variable for change in normalized vocabulary score (zvocab) and use it as the outcome for same regression specification as Item 5.

In [ ]:
change_in_vocab = multiple_shocks_table.column('zvocab_12') - multiple_shocks_table.column('zvocab_8')
multiple_shocks_table = multiple_shocks_table.with_column('ch_vocab', change_in_vocab).where(change_in_vocab==change_in_vocab)
multiple_shocks_table

In [ ]:
outcome = multiple_shocks_table.column('ch_vocab')
predictors = multiple_shocks_table.select(np.append(other_shocks, make_array('drought', 'wi_8'))).to_array()

results = sm.OLS(outcome, predictors).fit()
results.summary()